In [1]:
import os
from pathlib2 import Path
import re
import itertools
import copy

import pyarrow.feather as feather
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
category_data = feather.read_feather(Path('IceCat_Cat_2833_feather/frame_IceCat_Category_194.feather'))
display(category_data.tail())
category_data.info()

,level_0,index,id,name,category_id,category_label,Numeric keypad.1006,Numeric keypad.1006.unit,Hot keys.1007,Hot keys.1007.unit,...,Switching point.36686,Switching point.36686.unit,Adapters included.16912,Adapters included.16912.unit,Response time.1516,Response time.1516.unit,Driver requirement.32990,Driver requirement.32990.unit,Box contents.3421,Box contents.3421.unit
11258,11259,0,IceCat_Prod_28517486,Kit Standard Keyboard and Mouse Wireless 2.4GH...,IceCat_Category_194,Keyboards,False,None,<NA>,None,...,None,None,None,None,NaN,None,None,None,None,None
11259,11260,0,IceCat_Prod_11769927,Wired Desktop 600,IceCat_Category_194,Keyboards,<NA>,None,<NA>,None,...,None,None,None,None,NaN,None,None,None,None,None
11260,11261,0,IceCat_Prod_21081619,KB410,IceCat_Category_194,Keyboards,False,None,<NA>,None,...,None,None,None,None,NaN,None,None,None,None,None
11261,11262,0,IceCat_Prod_14742284,"KB410, PS/2",IceCat_Category_194,Keyboards,False,None,<NA>,None,...,None,None,None,None,NaN,None,None,None,None,None
11262,11263,0,IceCat_Prod_22263899,KB-1400S,IceCat_Category_194,Keyboards,False,None,False,None,...,None,None,None,None,NaN,None,None,None,None,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11263 entries, 0 to 11262
Columns: 654 entries, level_0 to Box contents.3421.unit
dtypes: boolean(84), float64(130), int64(2), object(438)
memory usage: 50.8+ MB


In [16]:
pct_complete = (len(category_data) - category_data.isnull().sum()) / len(category_data)
missing_vals = pd.DataFrame({'col': category_data.columns,
                             'pct_complete': pct_complete})
missing_vals[missing_vals['pct_complete'] >= .75]


,col,pct_complete
level_0,level_0,1.000000
index,index,1.000000
id,id,1.000000
name,name,1.000000
category_id,category_id,1.000000
category_label,category_label,1.000000
Product colour.1766,Product colour.1766,0.966616
Mouse included.2944,Mouse included.2944,0.981710
Device interface.8510,Device interface.8510,0.965373
Connectivity technology.1655,Connectivity technology.1655,0.789665


In [27]:
cols_of_interest = missing_vals[missing_vals['pct_complete'] >= .75].iloc[:,0]
category_data[category_data.columns.intersection(cols_of_interest)]

,level_0,index,id,name,category_id,category_label,Product colour.1766,Mouse included.2944,Device interface.8510,Connectivity technology.1655,Keyboard form factor.8568,Keyboard style.8575
0,0,0,IceCat_Prod_87093963,Multi-Device Dual Wireless Compact Keyboard,IceCat_Category_194,Keyboards,Black,False,Bluetooth,None,None,None
1,1,0,IceCat_Prod_17364890,709694-181,IceCat_Category_194,Keyboards,Black,False,USB,Wired,Full-size (100%),Straight
2,2,0,IceCat_Prod_27290927,KEY1001NO,IceCat_Category_194,Keyboards,Black,False,USB,Wired,Full-size (100%),Straight
3,3,0,IceCat_Prod_16669644,C6000,IceCat_Category_194,Keyboards,Black,False,RF Wireless,Wireless,Full-size (100%),Straight
4,4,0,IceCat_Prod_32902322,KM6150,IceCat_Category_194,Keyboards,Black,False,USB,Wired,Full-size (100%),Straight
...,...,...,...,...,...,...,...,...,...,...,...,...
11258,11259,0,IceCat_Prod_28517486,Kit Standard Keyboard and Mouse Wireless 2.4GH...,IceCat_Category_194,Keyboards,Black,False,None,Wireless,Full-size (100%),Straight
11259,11260,0,IceCat_Prod_11769927,Wired Desktop 600,IceCat_Category_194,Keyboards,Black,False,USB,Wired,Full-size (100%),Straight
11260,11261,0,IceCat_Prod_21081619,KB410,IceCat_Category_194,Keyboards,Black,False,USB,Wired,Full-size (100%),Straight
11261,11262,0,IceCat_Prod_14742284,"KB410, PS/2",IceCat_Category_194,Keyboards,Black,False,PS/2,Wired,Full-size (100%),Straight


In [51]:
tree = ET.parse('IceCat Specifications/CategoriesList.xml')
root = tree.getroot()

In [52]:
# Get all categories from specification
category_parent_list = []
for category in root[0][0].findall('Category'):
    cat_id = category.attrib
    parent_id = category.find('ParentCategory').attrib
    category_parent_list.append((int(cat_id["ID"]), int(parent_id["ID"])))

# Construct tree of categories from specification
all_nodes = {n[0]: {} for n in category_parent_list}
tree = {}
for item in category_parent_list:
    id, parent = item
    if parent != 1:
        all_nodes[parent][id] = all_nodes[id]
    else:
        tree[id] = all_nodes[id]




In [53]:
# Get categories from provided feather files
feather_file_names = os.listdir("IceCat_Cat_2833_feather")
feather_categories = [int(re.search(r'[0-9]+', file_name).group(0)) for file_name in feather_file_names]


def dict_key_filter(obj, obj_filter):
    '''
    Filters dictionary to only include `obj` keys that are included in `obj_filter`. 
    https://stackoverflow.com/questions/31710271/how-to-filter-by-keys-through-a-nested-dictionary-in-a-pythonic-way
    '''
    def inner_dict_key_filter(obj): return dict_key_filter(obj, obj_filter)
    def to_keep(subtree): return not isinstance(subtree, (dict, list)) or subtree

    def build_subtree(key, value):
        if key in obj_filter:
            return copy.deepcopy(value) # keep the branch
        elif isinstance(value, (dict, list)):
            return inner_dict_key_filter(value) # continue to search
        return [] # just an orphan value here

    if isinstance(obj, dict):
        key_subtree_pairs = ((key, build_subtree(key, value)) for key, value in obj.items())
        return {key:subtree for key, subtree in key_subtree_pairs if to_keep(subtree)}
    elif isinstance(obj, list):
        return list(filter(to_keep, map(inner_dict_key_filter, obj)))
    return []


filtered_tree = dict_key_filter(tree, feather_categories)




In [56]:
filtered_tree

{2833: {2: {7: {},
   12: {},
   21: {24: {},
    25: {},
    31: {},
    32: {},
    44: {},
    47: {},
    59: {},
    89: {},
    95: {},
    98: {},
    99: {},
    886: {},
    887: {},
    900: {},
    1487: {},
    1517: {},
    8745: {},
    9060: {}},
   26: {},
   27: {},
   28: {},
   38: {},
   42: {},
   55: {4: {},
    51: {},
    56: {},
    57: {},
    58: {},
    60: {},
    63: {},
    65: {},
    66: {},
    7243: {}},
   62: {},
   67: {73: {},
    80: {},
    82: {},
    87: {},
    88: {},
    872: {},
    875: {},
    878: {},
    1057: {},
    1504: {},
    2446: {},
    2738: {}},
   803: {},
   836: {},
   885: {},
   893: {2621: {}, 2622: {}},
   904: {},
   1534: {},
   1535: {},
   1552: {},
   1624: {},
   1646: {},
   1657: {},
   1719: {},
   1753: {},
   2444: {},
   2841: {},
   2842: {},
   2843: {14: {}, 15: {}, 1523: {}},
   3912: {},
   5210: {},
   5230: {},
   5231: {},
   5640: {},
   7100: {},
   7241: {},
   9225: {}},
  106: {166: {184: {},


In [60]:

def flatten(d):    
    res = []
    if isinstance(d, dict):
        for key, val in d.items():
            res.append(key)
            res.extend(flatten(val))
    return res

collapsed_cat_filtered_tree = flatten(filtered_tree)
len(collapsed_cat_filtered_tree)

562

In [84]:
filtered_tree[2833][2][55][60]

{}

In [85]:
cpy = copy.copy(collapsed_cat_filtered_tree)
cpy.sort()
len(cpy)

562

In [92]:
level_one_count = 0
level_two_count = 0
level_three_count = 0
level_four_count = 0
level_one_items  = []
level_two_items = []
level_three_items = []
level_four_items = []
for level_one in filtered_tree.keys():
    level_one_count += 1
    level_one_items.append(level_one)
    for level_two in filtered_tree[level_one].keys():
        level_two_count += 1
        level_two_items.append(level_two)
        for level_three in filtered_tree[level_one][level_two].keys():
            level_three_items.append(level_three)
            level_three_count += 1
            for level_four in filtered_tree[level_one][level_two][level_three].keys():
                level_four_count += 1
                level_four_items.append(level_four)



In [88]:
[level_one_count, level_two_count, level_three_count, level_four_count]

[1, 11, 212, 338]

In [91]:
level_three_items

[7,
 12,
 21,
 26,
 27,
 28,
 38,
 42,
 55,
 62,
 67,
 803,
 836,
 885,
 893,
 904,
 1534,
 1535,
 1552,
 1624,
 1646,
 1657,
 1719,
 1753,
 2444,
 2841,
 2842,
 2843,
 3912,
 5210,
 5230,
 5231,
 5640,
 7100,
 7241,
 9225,
 166,
 178,
 189,
 236,
 998,
 1291,
 1762,
 2702,
 2804,
 2844,
 2901,
 5984,
 6892,
 8839,
 8917,
 151,
 153,
 154,
 155,
 156,
 159,
 203,
 896,
 897,
 1062,
 1389,
 1508,
 1573,
 2093,
 2282,
 2774,
 2775,
 3319,
 3625,
 5022,
 5400,
 5459,
 5467,
 5632,
 5831,
 6946,
 7284,
 7291,
 7319,
 7396,
 8179,
 8189,
 8194,
 8231,
 8355,
 8738,
 8909,
 8910,
 8919,
 8921,
 9228,
 192,
 194,
 195,
 196,
 198,
 202,
 204,
 282,
 1041,
 1297,
 1540,
 2675,
 2813,
 2929,
 3955,
 8154,
 8470,
 8500,
 8603,
 8685,
 287,
 2840,
 8763,
 8764,
 373,
 692,
 1305,
 4972,
 4973,
 8653,
 2503,
 8752,
 8753,
 8754,
 8755,
 8756,
 8757,
 8758,
 867,
 869,
 883,
 952,
 953,
 956,
 1121,
 1158,
 1303,
 1306,
 1493,
 1494,
 1495,
 1501,
 1569,
 1574,
 1582,
 1614,
 1638,
 1639,
 1663,
 1

In [46]:
def flatten(d):    
    res = []  # Result list
    if isinstance(d, dict):
        for key, val in d.items():
            res.append(key)
            res.extend(flatten(val))
    # elif isinstance(d, list):
    #     res = d        
    # else:
    #     raise TypeError("Undefined type for flatten: %s"%type(d))

    return res


dict1 = {
    'Bob': {
        'shepherd': [4, 6, 3],
        'collie': [23, 3, 45],
        'poodle': [2, 0, 6],
    },
    'Sarah': {
        'shepherd': [1, 2, 3],
        'collie': [3, 31, 4],
        'poodle': [21, 5, 6],
    },
    'Ann': {
        'shepherd': [4, 6, 3],
        'collie': [23, 3, 45],
        'poodle': [2, 10, 8],
    }
}

print( flatten(dict1) )

['Bob', 'shepherd', 'collie', 'poodle', 'Sarah', 'shepherd', 'collie', 'poodle', 'Ann', 'shepherd', 'collie', 'poodle']


In [19]:
level_two_items

[2, 106, 150, 191, 206, 220, 225, 242, 830, 839, 2557]

In [13]:
len(res[2833])


NameError: name 'res' is not defined

In [ ]:
st ='frame_IceCat_Category_1007.feather'
re.search(r'[0-9]+', st).group(0)

'1007'